# Computación Avanzada y sus Aplicaciones a Ingeniería

### Máster Universitario en Ingeniería Informática


# Práctica 3 - Parte I - Machine Learning con Spark

En esta práctica veremos cómo utilizar las técnicas de Machine Learning disponibles en la librería Mlib de Apache Spark. Sigue detenidamente todos los bloques y prueba a cambiar los valores establecidos para comprobar su funcionamiento.

Ten en cuenta que una vez tengas en marcha Spark, podrás visualizar la evolución de cada trabajo de Spark en  <http://localhost:4040>

En caso de estar utilizando pySpark, **NO** es necesario inicializar el `SparkSession`, es decir, **no** ejecutar la siguiente celda

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Ejemplo pySparkSQL") \
    .config("spark.sql.warehouse.dir", "file:///D:/tmp/spark-warehouse") \
    .getOrCreate()

sc = spark.sparkContext


En caso de estar usando pySpark, ejecutar el siguiente comando o inciiar pyspark con 

`pyspark --conf spark.sql.warehouse.dir=file:///D:/tmp/spark-warehouse`

In [ ]:
spark.conf.set("spark.sql.warehouse.dir", "file:///D:/tmp/spark-warehouse")

Otros imports necesarios:

In [2]:
%matplotlib inline 
import matplotlib.pyplot as plt
from test_helper import Test
from pyspark.sql.functions import *
from pyspark.sql import Row

In [4]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression

# Ejemplos de código con LogisticRegression
Vamos a entrenar un modelo de regresión logística con MLlib

In [5]:
# Creamos unos datos de entrenamiento como una lista de tuplas (label, features).
training = spark.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, -1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, -0.5]))], ["label", "features"])

# Creamos una instancia de LogisticRegression. Esta instancia es un Estimator.
lr = LogisticRegression(maxIter=10, regParam=0.01)
# Imprimimos los parámetros, documentación y valores por defecto.
print "LogisticRegression parameters:\n" + lr.explainParams() + "\n"

# Aprendemos un modelo LogisticRegression. Utiliza los parámetros almacenados en lr.
model1 = lr.fit(training)


LogisticRegression parameters:
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
maxIter: max number of iterations (>= 0). (default: 100, current: 10)
predictionCol: prediction column name. (default: prediction)
probabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities. (default: probability)
rawPredictionCol: raw prediction (a.k.a. confidence) column name. (default: rawPrediction)
regParam: regularization parameter (>= 0). (default: 0.0, current: 0.01)
standardization: whether to standardize the training features before fitting the model. (default: Tr

## Otra forma de especificación de parámetros

In [7]:

# También podemos especificar los parámetros usando un dictionary de Python como paramMap
paramMap = {lr.maxIter: 20}
paramMap[lr.maxIter] = 30  # Especificamos 1 Param, sobrescribiendo el maxIter original.
paramMap.update({lr.regParam: 0.1, lr.threshold: 0.55})  # Especificamos múltiples parámetros

# Podemos combinar paramMaps, son diccionarios de Python
paramMap2 = {lr.probabilityCol: "myProbability"}  # Cambiamos el nombre de la columna de salida
paramMapCombined = paramMap.copy()
paramMapCombined.update(paramMap2)

# Aprendemos un nuevo modelo usando los parámetros en paramMapCombined.
# paramMapCombined sobrescribe todos los parámetros establecidos anteriormente con lr.set*
model2 = lr.fit(training, paramMapCombined)


## Obtenemos los resultados en test

In [8]:
# Preparamos el conjunto de test
test = spark.createDataFrame([
    (1.0, Vectors.dense([-1.0, 1.5, 1.3])),
    (0.0, Vectors.dense([3.0, 2.0, -0.1])),
    (1.0, Vectors.dense([0.0, 2.2, -1.5]))], ["label", "features"])

# Obtenemos las predicciones sobre los datos de test usando Transformer.transform()
# LogisticRegression.transform solo usa la columna llamada features
# El método model2.transform() devuelve una columna "myProbability" column en vez de 
# 'probability' ya que hemos cambiado el parámetro lr.probabilityCol 
prediction = model2.transform(test)
selected = prediction.select("features", "label", "myProbability", "prediction")
for row in selected.collect():
    print row


Row(features=DenseVector([-1.0, 1.5, 1.3]), label=1.0, myProbability=DenseVector([0.0571, 0.9429]), prediction=1.0)
Row(features=DenseVector([3.0, 2.0, -0.1]), label=0.0, myProbability=DenseVector([0.9239, 0.0761]), prediction=0.0)
Row(features=DenseVector([0.0, 2.2, -1.5]), label=1.0, myProbability=DenseVector([0.1097, 0.8903]), prediction=1.0)


# Ejemplos de código con LogisticRegression: Pipeline
Vamos a entrenar un modelo de regresión logística con MLlib. En este caso, haremos todo el proceso mediante una pipeline, incluyendo la creación de características.

In [10]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

# Preparamos los documentos de entrenamiento a partir de una lista de tuplas (id, text, label)
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)], ["id", "text", "label"])

# Configuramos una pipeline de ML, que consiste en tres etapas: tokenizer, hashingTF, and lr.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.01)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

# Entrenamos la pipeline con los documentos
model = pipeline.fit(training)

## Obtenemos los resultados en test

In [11]:
# Preparamos el conjunto de test con documentos no etiquetados (id, text)
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "mapreduce spark"),
    (7, "apache hadoop")], ["id", "text"])

# Hacemos las predicciones sobre el test
prediction = model.transform(test)
selected = prediction.select("id", "text", "prediction")
for row in selected.collect():
    print(row)

Row(id=4, text=u'spark i j k', prediction=0.0)
Row(id=5, text=u'l m n', prediction=0.0)
Row(id=6, text=u'mapreduce spark', prediction=0.0)
Row(id=7, text=u'apache hadoop', prediction=0.0)


# Selección de modelos / ajuste de parámetros
Spark permite llevar acabo la selección de parámetros de los algoritmos mediante dos formas:
* CrossValidation
* Train/Validation split


## Selección de modelos con CrossValidator

In [12]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Preparamos los documentos de entrenamiento etiquetados.
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0),
    (4, "b spark who", 1.0),
    (5, "g d a y", 0.0),
    (6, "spark fly", 1.0),
    (7, "was mapreduce", 0.0),
    (8, "e spark program", 1.0),
    (9, "a e c l", 0.0),
    (10, "spark compile", 1.0),
    (11, "hadoop software", 0.0)
], ["id", "text", "label"])

# Configuramos la pipeline, que consiste de tres etapas:  tokenizer, hashingTF, and lr.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

CrossValidator requiere de un estimador, un conjunto de parámetros sobre los que realizar la búsqueda y un evaluador. Además, hay que indicar el número de particiones a utilizar.

In [ ]:
# Tratamos la Pipeline como un Estimator para incluirla en una instancia de CrossValidator.
# De esta forma podemos optimizar los parámetros de todas las fases de manera conjunta.
# Un CrossValidator require de un Estimator, un conjunto de ParamMaps del Estimator y un Evaluator.
# Hacemos usod del ParamGridBuilder para construir el grid de parámetros.
# Usamos 3 valores para hashingTF.numFeatures y 2 valores para lr.regParam,
# Este grid tendrá 3 x 2 = 6 combinaciones de las que se elegirá el modelo.
paramGrid = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [10, 100, 1000]) \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=2)  # usar más de 3 particiones en la práctica

# Ejecutamos la validación cruzada, y elegimos el mejor conjunto de parámetros.
cvModel = crossval.fit(training)

In [ ]:
# Preparamos los documentos de test sin etiqueta
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "mapreduce spark"),
    (7, "apache hadoop")
], ["id", "text"])

# Realizamos las predicciones sobre los documentos de test. cvModel  utiliza el mejor modelo encontrado.
prediction = cvModel.transform(test)
selected = prediction.select("id", "text", "prediction", "probability")
for row in selected.collect():
    print(row)

## Selección de modelos con TrainValidationSplit

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

# Preparamos los datos de entrenamiento y test.
data = spark.read.format("libsvm")\
    .load("sample_linear_regression_data.txt")
train, test = data.randomSplit([0.7, 0.3])
lr = LinearRegression(maxIter=10, regParam=0.1)

TrainValidationSplit requiere de un estimador, un conjunto de parámetros sobre los que realizar la búsqueda y un evaluador. Además, hay que indicar el tamaño del training respecto a la validación (trainRatio)

In [ ]:
# Usamos un ParamGridBuilder para construir el grid de parámetros sobre el que realizar la búsqueda.
# TrainValidationSplit probará todas las combinaciones de valores para quedarse con la mejor
paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()

# El estimador es la regresión lineal.
# Un TrainValidationSplit requiere de un Estimator, un conjunto de parámetros ParamMaps y un Evaluator.
tvs = TrainValidationSplit(estimator=lr,
                           estimatorParamMaps=paramGrid,
                           evaluator=RegressionEvaluator(),
                           # 80% of the data will be used for training, 20% for validation.
                           trainRatio=0.8)


# Ejecutamos TrainValidationSplit, y obtenemos la mejor combinación de parámetros.
model = tvs.fit(train)

In [ ]:
# Hacemos las predicciones en test. model es el modelo con la mejor combinación de parámetros.
prediction = model.transform(test)
for row in prediction.take(5):
    print(row)